### Input: XYZ file

Can grab .mol file from https://cccbdb.nist.gov/ (Composite G4 calculation) and converted .mol format to .xyz using OpenBabel

### Output: CIF

Outputs a CIF version of the .xyz file, with some cell size.

In [5]:
import os
from pymatgen.core import Molecule

def create_boxed_structure(molecule, a, b, c):
    try:
        # Attempt to create a boxed structure
        structure = molecule.get_boxed_structure(a, b, c)
        return structure, True
    except ValueError as e:
        if str(e) == "Box is not big enough to contain Molecule.":
            # If the error is due to box size, double the dimensions and try again
            return None, False
        else:
            # If it's a different error, re-raise the exception
            raise e

# Get a list of all files in the current directory
file_list = [file for file in os.listdir() if file.endswith(".xyz")]

# Iterate through each XYZ file and process it
for file_name in file_list:

    # Define initial cell dimensions in Angstroms
    a, b, c = 15, 15, 15
    
    for _ in range(10):  # Try doubling the box size up to 10 times
        try:
            # Read molecule from the XYZ file
            molecule = Molecule.from_file(file_name)
            
            # Attempt to create a boxed structure with current dimensions
            structure, success = create_boxed_structure(molecule, a, b, c)
            
            if success:
                # Construct the output CIF filename (same name as XYZ file but with .cif extension)
                output_file = os.path.splitext(file_name)[0] + ".cif"
                
                # Save the structure to the CIF file
                structure.to(filename=output_file)
                
                print(f"Conversion successful: {file_name} -> {output_file}")
                break  # Exit the loop if successful
            else:
                print(f"Trying Doubling box size for {file_name}, new size: {a}, {b}, {c}")
                # Double the box size for the next attempt
                a *= 2
                b *= 2
                c *= 2
                
        except Exception as e:
            print(f"Error processing {file}: {e}")

['CO2.xyz', 'ice_prd.644.03.f.xyz']
Conversion successful: CO2.xyz -> CO2.cif
Trying Doubling box size for ice_prd.644.03.f.xyz, new size: 15, 15, 15
Trying Doubling box size for ice_prd.644.03.f.xyz, new size: 30, 30, 30
Conversion successful: ice_prd.644.03.f.xyz -> ice_prd.644.03.f.cif
